In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
#%cd /content/drive/MyDrive/llm_mcq_bias-main


In [3]:
from baselines.bnp_wip import (
    ARCChallengeDataset,
    Llama3_8B,
    bias_vector_computation,
    bias_node_pruning,
    create_prompt,
    restore_model,
    create_permuted_prompts,

)

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

arc_ds = ARCChallengeDataset()

In [5]:
for data in arc_ds.loader(split="test", batch_size=1):
    prompts = create_prompt(data)
    print(prompts[0][0])
    break

You are an AI assistant that answers multiple choice questions. Please respond with capitalized alphabet(s) that correspond to the correct answer An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation? (A) Planetary density will decrease.. (B) Planetary years will become longer.. (C) Planetary days will become shorter.. (D) Planetary gravity will become stronger..  Output: 


In [6]:
model = Llama3_8B()


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.27it/s]


In [7]:
# bias_vector = bias_vector_computation(model, arc_ds, k=32)

In [8]:
import torch
# torch.save(bias_vector, "arcchallenge_qwen_bias.pt")
bias_vector= torch.load("arcchallenge_qwen_bias.pt")


In [9]:
# perform bnp
orig_weight = bias_node_pruning(model, bias_vector)

# restore_model(model,orig_weight)

In [ ]:
from typing import List

In [10]:
class Model:
    """A simple huggingface model wrapper that creates both the model and the tokenizer
    and enables access to both model and tokenizer attributes and methods.

    For example, working with the string "Which topic area is best for research" using
    Llama3-8B on gpu;
    >>> model = Model("meta-llama/Meta-Llama-3-8B", "cuda")
    >>> text = " Which topic area is best for research"
    >>> e_input = model.tokenizer_call(text, return_tensors="pt") # encode with tokenizer
    >>> outputs = model(**e_input.to(model.device)) # get_outputs
    """

    def __init__(self, model_id: str, device: str = "cuda") -> None:
        """Initializes the model and tokenizer.

        Args:
            model_id: The huggingface model id to load.
            device: The device to load the model on. Defaults to "cuda".
        """
        self.hf_model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto",
        )
        self.option_to_idx = {}
        self.option_to_idx_2 = {}
        self.idx_to_option_2 ={}

        self.hf_tokenizer = AutoTokenizer.from_pretrained(model_id)
        for label in ["A", "B", "C", "D", "E"]:
            self.option_to_idx[label] = self.hf_tokenizer.encode(label)[-1]

        for label in ["_A", "_B", "_C", "_D", "_E"]:
            self.option_to_idx_2[label] = self.hf_tokenizer.encode(label)[-1]

        self.idx_to_option = {v: k for k, v in self.option_to_idx.items()}
        self.idx_to_option_2 = {v: k for k, v in self.option_to_idx_2.items()}

        self.options = [k for k in self.idx_to_option.keys()]
        self.options_2 = [k for k in self.idx_to_option_2.keys()]

        torch.cuda.empty_cache()
        gc.collect()

    def model_call(self, *args, **kwargs):
        """Calls the huggingface model with the given arguments.

        Args:
            *args: The positional arguments to pass to the huggingface model.
            **kwargs: The keyword arguments to pass to the huggingface model.
        Returns:
            The output of the huggingface model.
        """
        return self.hf_model(*args, **kwargs)

    def tokenizer_call(self, *args, **kwargs):
        """Calls the huggingface tokenizer with the given arguments.

        Args:
            *args: The positional arguments to pass to the huggingface tokenizer.
            **kwargs: The keyword arguments to pass to the huggingface tokenizer.
        Returns:
            The output of the huggingface tokenizer.
        """
        return self.hf_tokenizer(*args, **kwargs)

    def get_answer(self, text: str, **kwargs) -> str:
        """ """
        e_input = self.tokenizer_call(text, return_tensors="pt").to(self.device)
        outputs = self.model_call(**e_input, **kwargs)
        return self.idx_to_option[
            self.options[outputs.logits[:, -1, self.options].argmax().item()]
        ]

    def get_answer_and_embedding(self, texts: List[str], **kwargs) -> str:
        """ """
        if isinstance(texts, str):
            texts = [texts]
        e_input = self.tokenizer_call(texts, return_tensors="pt").to(self.device)
        outputs = self.model_call(**e_input, **kwargs, output_hidden_states=True)
        opt_token_ids = (outputs.logits[:, -1, self.options]  + outputs.logits[:, -1, self.options_2]).argmax(dim=-1)
        result = []
        for i in range(len(texts)):
            result.append(
                (
                    self.idx_to_option[self.options[opt_token_ids[i].item()]],
                    outputs.hidden_states[-1][i, -1].detach().cpu(),
                )
            )
        return result

    def compute_bias(self, data, **kwargs)-> float:
        all_prompts, option_map = create_permuted_prompts(data, bias=True)
        e_input = self.tokenizer_call(all_prompts, return_tensors="pt").to(self.device)
        logits = self.model_call(**e_input, **kwargs).logits
        # print(logits)
        preds_probabs = torch.nn.functional.softmax(logits, dim=2)[:, -1, self.options]
        preds_probabs = preds_probabs/preds_probabs.sum(dim=1).unsqueeze(1)
        z = torch.tensor(option_map).cpu()
        reordered_probs = torch.gather(preds_probabs.cpu(), 1, torch.argsort(z, dim=1))
        bias = torch.mean(torch.var(reordered_probs, dim=0))

        return bias

    def __call__(self, *args, **kwargs):
        """Calls the huggingface model with the given arguments, ensuring that certain
        keyword arguments are set to a default value if not provided.

        Args:
            *args: The positional arguments to pass to the huggingface model.
            **kwargs: The keyword arguments to pass to the huggingface model.
        Returns:
            The output of the huggingface model.
        """
        # update keyword arguments with default values
        kwargs.update(
            {"output_hidden_states": kwargs.get("output_hidden_states", True)}
        )
        return self.model_call(*args, **kwargs)

    def __getattr__(self, name):
        """Calls the huggingface model or tokenizer attribute with the given name.

        Args:
            name: The name of the attribute to call.
        Returns:
            The output of the huggingface model or tokenizer attribute.
        """
        try:
            return getattr(self.hf_model, name)
        except AttributeError:
            return getattr(self.hf_tokenizer, name)
        raise AttributeError(f"Model has no attribute {name}")


class Llama3_8B(Model):
    model_id = "Qwen/Qwen2.5-3B-Instruct"

    def __init__(self, device="cuda"):
        # assert os.environ.get("HF_TOKEN", None), (
        #     "Please ensure that the HF_TOKEN environment variable is set with your"
        #     " huggingface hub token"
        # )
        # login(
        #     token=os.environ.get("HF_TOKEN")
        # )  # Loging to huggingface hub. You need to\
        # be autorized to use llama
        super().__init__(Llama3_8B.model_id, device)

NameError: name 'List' is not defined

In [ ]:
import torch
# Accuracy after bnp
num_examples = 0
num_correct = 0
with torch.inference_mode():
    model.eval()
    for data in arc_ds.loader(split="test", batch_size=1):
        prompts = create_prompt(data)
        answers = [p[1] for p in prompts]
        preds = [model.get_answer(p[0]) for p in prompts]
        bias = model.compute_bias(data)
        for pred, ans in zip(preds, answers):
            num_examples += 1
            if pred == ans:
                num_correct += 1
        print(f"\rAccuracy: {num_correct/num_examples:.4f} | Num Examples: {num_examples} | Num Correct: {num_correct}", end="")